In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "devkartik/gg_llm",
    # max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth unsuccessfully patched LoraLayer.update_layer. Please file a bug report.
Luckily, your training run will still work in the meantime!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


adapter_config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.7 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only and it must be a big response. Write a response that appropriately answers and elaborate the request.

### Input:
{}

### Response:
{}"""

In [ ]:
# alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.

# ### Input:
# {}

# ### Response:
# {}"""

# def extract_response(full_text):
#     # Assuming the response is after "### Response:\n"
#     full_text = full_text.replace("<|end_of_text|>","")
#     response_start = full_text.find("### Response:\n")
#     if response_start != -1:
#         return full_text[response_start + len("### Response:\n"):].strip()
#     return full_text

# def generate_response(input_text, model, tokenizer):
#     prompt = alpaca_prompt.format(input_text, "")
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = False)
#     output = tokenizer.batch_decode(outputs)
#     responses = set()
#     while len(responses) < 5:
#         response = generate_response(input_text, model, tokenizer)
#         if len(response.split()) >= 2:
#             responses.add(extract_response(output[0]))
#     return list(responses)

In [ ]:
def generate_response_with_stream(input_text, model, tokenizer):
    prompt = alpaca_prompt.format(input_text, "")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    output = tokenizer.batch_decode(outputs)
    return extract_response(output[0])

In [ ]:
def extract_response(full_text):
    full_text = full_text.replace("<|end_of_text|>","")
    response_start = full_text.find("### Response:\n")
    if response_start != -1:
        return full_text[response_start + len("### Response:\n"):].strip()
    return full_text

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
input_text = "What is the meaning of the name krsna?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Krsna means all-attractive.</s>


In [ ]:
input_text = "what is the crux of bhagwad gita?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

That we are not this body. This is our crux.</s>


In [ ]:
input_text = "What about gandhi's non-violence policies?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

There is violence. You are thinking that there is no violence, but it is there.</s>


In [ ]:
input_text = "What gita teaches us?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

That you are not this body. This is the first lesson.</s>


In [ ]:
input_text = "What about kalyuga?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Kalyuga?</s>


In [ ]:
input_text = "What can we do if we are in a state of sadness to come out of that state?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

You have to take lessons from Kṛṣṇa. That is Kṛṣṇa consciousness.</s>


In [ ]:
input_text = "Tell me something about life?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Life is the same.</s>


In [ ]:
input_text = "What is the maha mantra?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

HareKṛṣṇa,HareKṛṣṇa,KṛṣṇaKṛṣṇaHareHare/HareRāma,HareRāma,RāmaRāmaHareHare.</s>


In [ ]:
input_text = "tell me about iskcon?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Iskcon means International Society for Krishna Consciousness.</s>


In [ ]:
input_text = "why iskcon is started?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Iskcon is started for the service of the Lord. That's all.</s>


In [ ]:
input_text = "how can we gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

You have to give up this rascaldom. That is Kṛṣṇaconsciousness.</s>


In [ ]:
input_text = "what are the steps to gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

That is stated in theBhagavad-gītā.</s>


In [ ]:
input_text = "what are the steps to gain Kṛṣṇaconsciousness?"
response = generate_response_with_stream(input_text, model, tokenizer)
print(response)

Yes. First of all, you must accept the fact that you are not this body.</s>
